# Chapter 4 - Interest rates

## Imports

In [463]:
import math

## Problems

### 4.1

In [464]:
quarterly_rate = 0.07
quarterly_compounded = math.pow(1 + quarterly_rate / 4, 4)

#### Continuous compounding

In [465]:
continuous_rate = math.log(quarterly_compounded)
print("continuous rate: ", continuous_rate)

continuous rate:  0.06939455333845228


#### Annually compounded

In [466]:
annual_rate = quarterly_compounded - 1
print("annual rate: ", annual_rate)

annual rate:  0.07185903128906279


### 4.2

In [467]:
semi_annual_rate = 0.05
continuous_rate = math.log((1 + semi_annual_rate / 2) ** 2)
print("continuous zero rate: ", continuous_rate)

continuous zero rate:  0.049385225180742925


#### Bond price

In [468]:
bond_yield = 0.052
half_year_yield = math.exp(-bond_yield / 2)
bond_price = 2 * half_year_yield * (1 + half_year_yield) + 102 * (half_year_yield ** 3)
print("bond price: ", bond_price)

bond price:  98.1936994203443


#### 18 month (continuous) zero rate

In [469]:
half_year_cost = math.exp(continuous_rate / 2)
half_year = 2 * half_year_cost
full_year = 2 * half_year_cost ** 2
extended_zero_rate = -math.log((bond_price - half_year - full_year) / 102) / 1.5
print("18 month zero rate: ", extended_zero_rate)

18 month zero rate:  0.0541510289146814


### 4.3

In [470]:
yearly_rate = 0.1
print("yearly rate: ", yearly_rate)
semi_annual_rate = (math.sqrt(1 + yearly_rate) - 1) * 2
print("semi-annual rate: ", semi_annual_rate)
monthly_rate = (math.pow(1 + yearly_rate, 1 / 12) - 1) * 12
print("monthly rate: ", monthly_rate)
continuous_rate = math.log(1 + yearly_rate)
print("continuous rate: ", continuous_rate)

yearly rate:  0.1
semi-annual rate:  0.09761769634030326
monthly rate:  0.09568968514684517
continuous rate:  0.09531017980432493


### 4.4

#### Risk-free rates

In [471]:
three_month_rate = 0.03
print("three month rate: ", three_month_rate)
six_month_rate = 0.032
print("six month rate: ", six_month_rate)
nine_month_rate = 0.034
print("nine month rate: ", nine_month_rate)
twelve_month_rate = 0.035
print("twelve month rate: ", twelve_month_rate)
fifteen_month_rate = 0.036
print("fifteen month rate: ", fifteen_month_rate)
eighteen_month_rate = 0.037
print("eighteen month rate: ", eighteen_month_rate)

three month rate:  0.03
six month rate:  0.032
nine month rate:  0.034
twelve month rate:  0.035
fifteen month rate:  0.036
eighteen month rate:  0.037


#### Forward rates

##### The forward rate calculation

In [472]:
def forward_rate(previous_rate: float, previous_duration: float, next_rate: float, duration_difference: float) -> float:
    '''
    All durations are in years.  duration_difference is the next duration minus the previous duration.  E.g., 
    for the forward rates between quarters, this would be 3 months or 1/4 of a year.
    '''
    return next_rate + (previous_duration / duration_difference) * (next_rate - previous_rate)

print("The results of Table 4.5 are replicated below using forward_rate.")
print("year 1 -> year 2 forward rate: ", forward_rate(0.03, 1, 0.04, 1))
print("year 2 -> year 3 forward rate: ", forward_rate(0.04, 2, 0.046, 1))
print("year 3 -> year 4 forward rate: ", forward_rate(0.046, 3, 0.05, 1))
print("year 4 -> year 5 forward rate: ", forward_rate(0.05, 4, 0.053, 1))  

The results of Table 4.5 are replicated below using forward_rate.
year 1 -> year 2 forward rate:  0.05
year 2 -> year 3 forward rate:  0.057999999999999996
year 3 -> year 4 forward rate:  0.06200000000000001
year 4 -> year 5 forward rate:  0.06499999999999997


##### The quarterly forward rates

In [473]:
quarter_duration = 1 / 4
print("quarter conversion to years: ", quarter_duration)
print("====================================")

print("quarter 2 forward rate: ", forward_rate(three_month_rate, quarter_duration, six_month_rate, quarter_duration))
print("quarter 3 forward rate: ", forward_rate(six_month_rate, 2 * quarter_duration, nine_month_rate, quarter_duration))
print("quarter 4 forward rate: ", forward_rate(nine_month_rate, 3 * quarter_duration, twelve_month_rate, quarter_duration))
print("quarter 5 forward rate: ", forward_rate(twelve_month_rate, 4 * quarter_duration, fifteen_month_rate, quarter_duration))
print("quarter 6 forward rate: ", forward_rate(fifteen_month_rate, 5 * quarter_duration, eighteen_month_rate, quarter_duration))

quarter conversion to years:  0.25
quarter 2 forward rate:  0.034
quarter 3 forward rate:  0.038000000000000006
quarter 4 forward rate:  0.038000000000000006
quarter 5 forward rate:  0.03999999999999997
quarter 6 forward rate:  0.042


### 4.5

#### Convert rates to continuous

In [474]:
principal = 1000000
# From previous question, we want the quarter 5 forward rate
agreed_forward_rate = forward_rate(twelve_month_rate, 4 * quarter_duration, fifteen_month_rate, quarter_duration)
print("forward rate between the 12th and 15th months: ", agreed_forward_rate)
# Forgot originally: need to discount the payoff based on the zero rate (3.6% for 15 months, or after one quarter starting a year from now)
FRA_value = principal * (1 / 4) * (0.045 - agreed_forward_rate) * math.exp(-0.036 * 1.25)
print("FRA value to pay 4.5% and receive SOFR on $1,000,000: ", FRA_value)

forward rate between the 12th and 15th months:  0.03999999999999997
FRA value to pay 4.5% and receive SOFR on $1,000,000:  1194.996852291381


### 4.6

#### Upward sloping

If the term structure of interest rates is upward sloping, then the order of magnitudes is as follows from smallest to greatest  
- bond yield on 5-year coupon-bearing bond
- the 5 year zero rate
- the forward rate between 4.75 and 5 years

It's assumed that the upward-sloping nature is not pathologically small or localized at the 5 year mark. The price of the bond will be weighted by the coupon payouts in years 0-4, which are (by definition of upward sloping) lower interest than the 5 year mark when the principal is due.  As a result, the bond yield--calculated with a flat rate--will be lower than the 5 year zero rate.  The forward rate will be increased as it is necessarily larger than two rates involved, given that the interest curve is increasing between the two.  In this case, the two rates are at 4.75 and 5 years in the future, so the forward rate for this period is necessarily larger than the zero rate for 5 years.

#### Downward sloping

The trend is reversed.  The yield will be greatest as it is biased by larger rates near the start of the repayment period.  The zero rate will be less than the yield as the structure of interest rates is downward sloping.  The forward rate will be the least of all, as it is necessarily smaller than all terms involved for a downward sloping interest rate curve -- i.e., the forward rate between any earlier time and the 5 year period is necessarily smaller than both, which includes the 5 year rate.


### 4.7

The duration relates the sensitivity of the value of a bond-like investment to changes in the yield.  Because the yield likely shifts opposite the zero rates over the bond terms, the duration is a measure of how sensitive a bond's value is to change in the zero rates.  

This analysis is based on small variations due to small shifts in the yield (from which the interest rate is inferred).  For large shifts in any of these quantities, their linear relationship is no longer guaranteed; the trend may even be wrong in sign.

### 4.8

In [475]:
continuous_rate = math.log(math.pow((1 + 0.08 / 12), 12))
print("continuous compounding equivalent to 8% per annum compounded monthly: ", continuous_rate)

continuous compounding equivalent to 8% per annum compounded monthly:  0.07973451262402206


### 4.9

#### Note
I actually disagree with the book on this framing.  If the interest is compounded, then shouldn't the amount in each quarterly payout increase?

In [476]:
rate = 0.04
principal = 10000
quarter_compounded = math.exp(rate / 4)
first_quarter = principal * (quarter_compounded - 1)
print("first quarter payout: ", first_quarter)
second_quarter = (principal + first_quarter) * (quarter_compounded - 1)
print("second quarter payout: ", second_quarter)
third_quarter = (principal + first_quarter + second_quarter) * (quarter_compounded - 1)
print("third quarter payout: ", third_quarter)
fourth_quarter = (principal + first_quarter + second_quarter + third_quarter) * (quarter_compounded - 1)
print("fourth quarter payout: ", fourth_quarter)

first quarter payout:  100.5016708416795
second quarter payout:  101.51172942587641
third quarter payout:  102.53193926760932
fourth quarter payout:  103.56240238871256


### 4.10

In [477]:

principal = 100
coupon = 2
six_month = coupon * math.exp(-0.04 / 2)
twelve_month = coupon * math.exp(-0.042)
eighteen_month = coupon * math.exp(-0.044 * 3 / 2)
twentyfour_month = coupon * math.exp(-0.046 * 2)
thirty_month = (principal + coupon) * math.exp(-0.048 * 5 / 2)
price = six_month + twelve_month + eighteen_month + twentyfour_month + thirty_month
print("bond price estimate: ", price)

bond price estimate:  98.04049348058196


### 4.11

#### Solved using Wolframalpha
4 (exp[-y / 2] + exp[-y] + exp[-3 y / 2] + exp[-2 y] + exp[- 5 y / 2]) + 104 exp[-3 y ] == 104

#### Approximate yield
6.41%

### 4.12


In [478]:
six_month = math.exp(-0.05 / 2)
twelve_month = math.exp(-0.06)
eighteen_month = math.exp(-0.065 * 3 / 2)
twentyfour_month = math.exp(-0.07 * 2)
par_yield = 2 * 100 * (1 - twentyfour_month) / (six_month + twelve_month + eighteen_month + twentyfour_month)
print("par yield (in percent) for 2 year paid semi-annually: ", par_yield)

par yield (in percent) for 2 year paid semi-annually:  7.074077478783004


### 4.13

In [479]:
one_year = 0.02
two_year = 0.03
three_year = 0.037
four_year = 0.042
five_year = 0.045

print("forward rate between 1st and 2nd year: ", forward_rate(one_year, 1, two_year, 1))
print("forward rate between 2nd and 3rd year: ", forward_rate(two_year, 2, three_year, 1))
print("forward rate between 3rd and 4th year: ", forward_rate(three_year, 3, four_year, 1))
print("forward rate between 4th and 5th year: ", forward_rate(four_year, 4, five_year, 1))


forward rate between 1st and 2nd year:  0.039999999999999994
forward rate between 2nd and 3rd year:  0.051
forward rate between 3rd and 4th year:  0.057000000000000016
forward rate between 4th and 5th year:  0.05699999999999998


### 4.14

In [480]:
base = 100
years = 10
coupon_8_price = 90
coupon_4_price = 80
ten_year_rate = -math.log((2 * coupon_4_price - coupon_8_price) / base) / 10
print("ten year zero rate: ", ten_year_rate)

ten year zero rate:  0.03566749439387325


### 4.15

The liquidity preference theory of interest rates assumes that different rates for different loan durations are driven by opposing time preferences between lenders and borrowers.  All things being equal, borrowers are expected to prefer longer terms to lower their refinancing risk if rates go up and lenders are expected to prefer shorter terms to lower their opportunity cost if rates go up.  This results in an inbalanced demand for longer term loans from borrowers and an inbalanced supply for shorter term loans from lenders.  One expects, then, that lenders incentivize shorter-term loans with lower interest rates and lenders can charge higher interest rates for longer-term loans due to the increased demand.

### 4.16

An upward-sloping zero rate curve means that longer maturities pay higher interest rates.  The par yield is defined as the (semi-annual) coupon that compensates for the decreased present-value of the bond's principal at maturity.  These coupons are paid out at earlier times, where the present value is determined by the smaller, nearer-term zero rates between now and maturity.  As a result, the bond yield is partially weighted by rates lower than the one associated with the bond maturity.  

Conversely, a downward-sloping zero rate curve means that the bond yield is weighted by higher interest rates for nearer terms than the bond maturity.  As a result, the bond yield would exceed the zero rate at maturity in the case of a downward sloping zero rate.

### 4.17

Because a repo transaction is a secured loan, the lender values the assets it initially purchases at or above the amount extended to the borrower.  From the borrower's perspective, they wouldn't enter the repo transaction if their immediate opportunity didn't outweigh the payoff on the assets traded.  As a result, terms can be made between the two so that each side benefits even in the case of default.  The assets can be obtained at a sufficient bargain that the lender is in-the-money even if the borrower never buys them back.  Conversely, the initial purchase can be sufficiently near the asset value that the borrower is content with the cash they've received (and implied profit opportunities they've used it for).

### 4.18

#### Bond price

In [481]:
r = 0.07
f = math.exp(-r)
series = [8 * pow(f, i) for i in range(1, 5)]  # [1, 4], all coupons before last
price = sum(series) + 108 * pow(f, 5)
print("bond price: ", price)

bond price:  103.05127403146639


#### Bond duration

In [482]:
weighted_series = [8 * i * pow(f, i) for i in range (1, 5)] # [1, 4]
weighted_duration = sum(weighted_series) + 108 * 5 * pow(f, 5)
duration = weighted_duration / price
print("duration: ", duration)

duration:  4.323474470267659


#### Bond response to change in yield

In [483]:
delta_yield = -0.002
delta_price = - price * duration * delta_yield
print("change in bond price to yield drop of 0.02%: ", delta_price)

change in bond price to yield drop of 0.02%:  0.891079104807203


#### Bond price re-calcualted with new yield

In [484]:
r = 0.068
f = math.exp(-r)
series = [8 * pow(f, i) for i in range(1, 5)]  # [1, 4], all coupons before last
new_price = sum(series) + 108 * pow(f, 5)
print("re-calculated bond price: ", new_price)
print("Difference: ", new_price - price)
print("re-calculated price / estimated change: ", (new_price - price) / delta_price)

re-calculated bond price:  103.94655276510635
Difference:  0.8952787336399552
re-calculated price / estimated change:  1.0047129697128976


### 4.19

In [485]:
six_month_rate = 2 * math.log(100/94)
print("six month zero rate: ", six_month_rate)
twelve_month_rate = math.log(100/89)
print("twelve month zero rate: ", twelve_month_rate)
eighteen_month_rate = -(2 / 3) * math.log((94.84 - 4 * math.exp(-six_month_rate / 2) - 4 * math.exp(-twelve_month_rate)) / 104)
print("eighteen month zero rate: ", eighteen_month_rate)
twentyfour_month_rate = -(1 / 2) * math.log((97.12 - 5 * math.exp(-six_month_rate / 2) - 5 * math.exp(-twelve_month_rate) - 5 * math.exp(-eighteen_month_rate * 3 / 2)) / 105)
print("twentyfour month rate: ", twentyfour_month_rate)

six month zero rate:  0.1237508074361749
twelve month zero rate:  0.1165338162559516
eighteen month zero rate:  0.11501570697846777
twentyfour month rate:  0.11298861636472714


### 4.20

A five year interest rate swap for 6-month LIBOR in exchange for a fixed rate on a known principal involves an initial transaction followed by 9 FRAs, whose value is established every six months.  Since the 6-month LIBOR is (presumably) known at the time of transaction, the first sixth month interest rate swap has a known value.  The remaining 9 semi-annual payouts equate to FRAs.  In the first FRA, for example, LIBOR based on the next six months is exchanged for 5% on the known principal for the period between 6 months and 1 year from now.  This pattern continues for the period between 1 year and 18 months, 18 months and 2 years, and so on until the period between 4.5 and 5 years from now.

### 4.21

In [486]:
annual_rate = 0.011
def rate_converted_from_annual(periods_per_year: float) -> float:
    return periods_per_year * (math.pow(1 + annual_rate, 1 / periods_per_year) - 1)

print("rate compounded annually: ", annual_rate)
print("semi-annual compounding: ", rate_converted_from_annual(2))
print("quarterly compounding: ", rate_converted_from_annual(4))
print("monthly compounding: ", rate_converted_from_annual(12))
print("weekly compounding: ", rate_converted_from_annual(52))
print("daily compounding: ", rate_converted_from_annual(365))

rate compounded annually:  0.011
semi-annual compounding:  0.010969915239907202
quarterly compounding:  0.010954913972389235
monthly compounding:  0.010944928316093616
weekly compounding:  0.010941090910272067
daily compounding:  0.010940103988298366


### 4.22

In [487]:
price = 20 * (math.exp(-0.02 / 2) + math.exp(-0.023) + math.exp(-0.027 * 3 / 2)) + 1020 * math.exp(-0.032 * 2)
print("bond price: ", price)

bond price:  1015.3175291620212


#### Using Wolframalpha, solve below
20 * (exp[-y / 2] + exp[-y] + exp[-3 y / 2]) + 1020 * exp[-2 y] == 1015.3175

#### yield = 0.0318

### 4.23

#### Using Wolframalpha, solve below
2.5 * (exp[-0.5 y] + exp[-y] + exp[-1.5 y] + exp[-2 y] + exp[-2.5 y] + exp[-3 y] + exp[-3.5 y] + exp[-4 y] + exp[-4.5 y]) + 102.5 * exp[-5 y] == 104

#### yield = 0.0407

### 4.24

In [488]:
semi_annual_rate = 0.011
def rate_converted_from_semi_annual(periods_per_half_year: float) -> float:
    compounded_semi_annually = math.pow(1 + semi_annual_rate / 2, 2)
    return periods_per_half_year * (math.pow(compounded_semi_annually, 1 / periods_per_half_year) - 1)

print("interest rate with semi-annual compounding: ", semi_annual_rate)
print("annual compounding: ", rate_converted_from_semi_annual(0.5))
print("monthly compounding: ", rate_converted_from_semi_annual(6))
print("continuous compounding: ", math.log(math.pow(1 + semi_annual_rate / 2, 2)))

interest rate with semi-annual compounding:  0.011
annual compounding:  0.011091083207531383
monthly compounding:  0.01097989472866967
continuous compounding:  0.0109698604611395


### 4.25

In [489]:
first_price = 2000 * math.exp(-0.1) + 6000 * math.exp(-1)
print("price of first portfolio: ", first_price)
first_duration = (2000 * 1 * math.exp(-0.1) + 6000 * 10 * math.exp(-1)) / first_price
print("duration of first portfolio: ", first_duration)
print("-----------------")
print("Because the second portfolio is a single, zero-coupon bond, its duration is equal to its maturity of 5.95 years.")

price of first portfolio:  4016.9514831005736
duration of first portfolio:  5.945414428536803
-----------------
Because the second portfolio is a single, zero-coupon bond, its duration is equal to its maturity of 5.95 years.


In [490]:
first_delta = - 0.001 * first_duration
print("First relative change to small yield: ", first_delta)
second_delta = -0.001 * 5.95
print("Second relative change to small yield: ", second_delta)


First relative change to small yield:  -0.005945414428536803
Second relative change to small yield:  -0.00595


In [491]:
new_yield = 0.15
first_price_new = 2000 * math.exp(-new_yield) + 6000 * math.exp(-10 * new_yield)
first_price_delta = first_price_new - first_price
print("relative change in first portfolia to 5% yield increase: ", first_price_delta / first_price)
second_price = 5000 * math.exp(-0.05 * 5.95)
second_price_new = 5000 * math.exp(-new_yield * 5.95)
print("relative change in second portfolio: ", (second_price_new - second_price)/ second_price)

relative change in first portfolia to 5% yield increase:  -0.23817926937504028
relative change in second portfolio:  -0.4484374341321702


Notably, the relative price changes of two portfolios with the same duration are no longer equal (i.e., no longer linear in the duration) for significant parallel shifts in the yield.

### 4.26

#### Left out
I'm not too interested in playing with the DerivaGem software and I think the previous exercises cover durations vs. actual change sufficiently well.